## Re-examine SF_Uncert metric in WFD regarding rolling strategy 

In [1]:
# development code
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import glob
import os, sys
import itertools

In [3]:
# add the path the scripts
sys.path.insert(0, "../scripts/")

In [4]:
# import rubin_sim python modules
import rubin_sim.maf.db as db
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.stackers as stackers
import rubin_sim.maf.plots as plots
import rubin_sim.maf.metricBundles as metricBundles
import rubin_sim.maf as maf
from rubin_sim.maf.utils import m52snr

# footprint utils
from rubin_sim.scheduler.utils import footprints

# import convenience functions
from opsimUtils import *

# import custom stacker/metrics from script
from agnstructure import *

# print version
import rubin_sim

rubin_sim.__version__

'0.10.1.dev64+gc32f68b'

### 1. Pull rolling opsims to evaluate 

In [5]:
families = maf.archive.get_family_descriptions()
family_list = families.index.values

# pull roll runs
fam = ["rolling"]
baseline_run = families.loc["rolling", "reference"]
roll_runs = np.concatenate([[baseline_run], families.explode("run").loc[fam, "run"]])
roll_runs

array(['baseline_v2.0_10yrs', 'noroll_v2.0_10yrs',
       'rolling_ns2_rw0.5_v2.0_10yrs', 'rolling_ns3_rw0.5_v2.0_10yrs',
       'rolling_ns2_rw0.9_v2.0_10yrs', 'rolling_ns3_rw0.9_v2.0_10yrs',
       'rolling_bulge_ns2_rw0.5_v2.0_10yrs',
       'rolling_bulge_ns2_rw0.8_v2.0_10yrs',
       'rolling_bulge_ns2_rw0.9_v2.0_10yrs',
       'rolling_all_sky_ns2_rw0.9_v2.0_10yrs', 'roll_early_v2.0_10yrs',
       'six_rolling_ns6_rw0.5_v2.0_10yrs',
       'six_rolling_ns6_rw0.9_v2.0_10yrs', 'rolling_bulge_6_v2.0_10yrs'],
      dtype=object)

### 2. Run different combos of configs

In [6]:
dbDir = rubin_sim.data.get_data_dir()
outDir = "/home/jovyan/mount/ResearchData/MAFOutput/wfd_tgap/ResultsDb"
metricDataPath = "/home/jovyan/mount/ResearchData/MAFOutput/wfd_tgap/MetricData"

if not os.path.exists(os.path.abspath(outDir)):
    os.makedirs(os.path.abspath(outDir))

if not os.path.exists(os.path.abspath(metricDataPath)):
    os.makedirs(os.path.abspath(metricDataPath))

In [7]:
# create dict for storing dbs
opSimDbs, resultDbs = connect_dbs(dbDir, outDir, dbRuns=roll_runs)

In [8]:
# define combos of configs
nbins = [10, 15]
snr_cuts = [0, 3, 5]
combos = list(itertools.product(nbins, snr_cuts))

In [9]:
agn_m5 = {"u": 22.89, "g": 23.94, "r": 23.5, "i": 22.93, "z": 22.28, "y": 21.5}

#### 2.1 Use big_dust healpix cut

In [ ]:
# metric name template
metricNameTemp1 = 'logTgapDust_{}_{}_{}'
bdict1 = {}

# custom slicer
nside = 64
wfd_bigdust = footprints.WFD_no_dust_healpixels(nside, dec_max=12.25, dec_min=-72.5, 
                                                dust_limit=0.199)
wfd_slicer = slicers.HealpixSubsetSlicer(nside, np.where(wfd_bigdust==1)[0], 
                                         useCache=False)
wfd_slicer.slicerName = 'HealpixSlicer'  # hack maf


for run in roll_runs:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    bdict1 = {}

    # loop through band and config combos
    for band in 'ugrizy':
        constraint = f'filter = "{band}"' + ' and note not like "DD%"'
        for combo in combos:
            nbin, snr_cut = combo
            metricName = metricNameTemp1.format(band, nbin, snr_cut)

            # metric
            mybins = np.logspace(0, np.log10(3650), nbin+1)
            myweights = np.full(nbin, 1/nbin)
            sfuncertdev = SFUncertMetricDev(mag=agn_m5[band], 
                                            bins=mybins, weight=myweights, 
                                            snr_cut=0, metricName=metricName)
            # add bundle
            bdict1[metricName] = metricBundles.MetricBundle(sfuncertdev, wfd_slicer, 
                                                            constraint,
                                                            runName = run)

    gp = metricBundles.MetricBundleGroup(bdict1, opSimDbs[run], outDir=metricDataPath, 
                                         resultsDb=resultDbs[run], verbose=False)
    gp.runAll()

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Runing metrics on Opsim: baseline_v2.0_10yrs!
*****************************************


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubi

Runing metrics on Opsim: noroll_v2.0_10yrs!
*****************************************


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubi

Runing metrics on Opsim: rolling_ns2_rw0.5_v2.0_10yrs!
*****************************************


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubi

Runing metrics on Opsim: rolling_ns3_rw0.5_v2.0_10yrs!
*****************************************


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/maps/dustMap.py:42: UserWarning: Slicer value of nside 64 different from map value 128, using slicer value
  warnings.warn(


#### 2.2 SNR cut

In [ ]:
# metric name template
metricNameTemp2 = 'logTgapSNR_{}_{}_{}'
bdict2 = {}

# custom slicer
nside = 64
all_slicer = slicers.HealpixSlicer(nside, useCache=False)

for run in roll_runs:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************') 
    bdict2 = {}

    # loop through band and config combos
    for band in 'ugrizy':
        constraint = f'filter = "{band}"' + ' and note not like "DD%"'
        for combo in combos:
            nbin, snr_cut = combo
            metricName = metricNameTemp2.format(band, nbin, snr_cut)

            # metric
            mybins = np.logspace(0, np.log10(3650), nbin+1)
            myweights = np.full(nbin, 1/nbin)
            sfuncertdev = SFUncertMetricDev(mag=agn_m5[band],
                                            bins=mybins, weight=myweights, 
                                            snr_cut=snr_cut)
            # add bundle
            bdict2[metricName] = metricBundles.MetricBundle(sfuncert, all_slicer, 
                                                            constraint,
                                                            runName = run)

    gp = metricBundles.MetricBundleGroup(bdict2, opSimDbs[run], outDir=metricDataPath, 
                                         resultsDb=resultDbs[run], verbose=False)
    gp.runAll()